In [1]:
import matplotlib.pyplot as plt
import time


import sys
sys.path += ['Package/']  ### Specify the directory where the Package is
from functions_for_DDCSBM import *
from dynamic_SC_BH import *
from dynamic_SC import *
from static_SC import *

import warnings
warnings.filterwarnings("ignore")

## Figure 9

In [2]:
n = 2500
n_clusters = 2
n_average = 3
fluctuation = 0
fraction = np.ones(n_clusters)/n_clusters
n_methods = 5
T = 4 #number of timeslots
c = 6 #average degree
n_step = 15
eta = 0.7
theta = np.ones(n)
phi = 1

c_out_v = np.linspace(.5,5,n_step)

In [ ]:
t0 = time.time()
ov = np.zeros(((n_methods,n_average,n_step)))
for j in range(n_average):
    for i in range(n_step):
        c_out = c_out_v[i]
        c_in = n_clusters*c - (n_clusters-1)*c_out
        lamb = (c_in - c_out)/(n_clusters*c)
        alpha = np.sqrt(c*phi)*lamb/find_transition(T,eta)
        C = matrix_C(c_out, c, fluctuation, fraction)
        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
        cluster = dynamic_community_detection(AT, eta, real_classes = Label, n_clusters = n_clusters)
        ov[0][j][i] += np.mean(cluster.overlap)
        over, mod = dynamic_A(AT, n_clusters, Label)
        ov[1][j][i] += np.mean(over)
        if alpha > 1:
            over, mod = dynamic_B(AT, lamb, eta, n_clusters, Label)
            ov[2][j][i] += np.mean(over)
        over, mod = static_BH(AT,n_clusters, Label)
        ov[3][j][i] += np.mean(over)
        over, mod, k_esy = dynamic_louvain(AT, Label, eta)
        ov[4][j][i] += np.mean(over)
        
        

        
        OUT = 'i : ' + str(i+1) + '; j : ' + str(j+1) + '; time : ' + str(np.round(time.time()-t0,1))
        sys.stdout.write('\r%s' % OUT)
        sys.stdout.flush()
        
mean = np.zeros((n_methods,n_step))
var = np.zeros((n_methods,n_step))
for i in range(n_methods):
    mean[i] = np.sum(ov[i], axis = 0)/n_average
    var[i] = np.sum(ov[i]**2,axis = 0)/n_average - mean[i]**2

i : 3; j : 1; time : 60.4

### Note 

Louvain algorithm typically identifies a much larger number of communities that the planted ones but the computation of the overlap is made directly on the estimated label vector.

In [ ]:
fig = plt.figure(figsize = (10,6))



color = ['darkred', 'darkgreen', 'darkorange', 'fuchsia', 'k']
label = ['Alg 1', 'Dyn A', 'Dyn B', 'Static BH', 'dyn Louvain']
marker = ['o', 'D', 'P', '*', 'd']


c_in_v = n_clusters*c - (n_clusters-1)*c_out_v
lambda_v = (c_in_v-c_out_v)/(n_clusters*c)
alpha_v = np.sqrt(c*phi*lambda_v**2)/(find_transition(T,eta))

for i in range(n_methods):
    plt.errorbar(alpha_v,mean[i],np.sqrt(var[i]), marker = marker[i],markersize = 6, fillstyle = 'none', markeredgewidth = 1, color = color[i], linewidth = 1, label = label[i])
    
plt.axvline(1, color = 'k', linestyle = '--', linewidth = 0.8)
plt.legend()


plt.show();

## Figure 10

Here we report the codes for using the "optimal" version of $H_{\lambda,\eta}$ and $B_{\lambda,\eta}$. These codes are not included in the package as they are written explicitely for this application.

In [ ]:
def BH_opt(AT,n_clusters, Label, eta, lamb):

    T = len(AT) # number of time frames
    n = len(AT[0][0].A.T) # number of nodes
    real_classes = Label

    H = BH_matrix(AT,n,lamb,eta) # build the Bethe-Hessian matrix
    info, X = scipy.sparse.linalg.eigsh(H, k=n_clusters*T, which='SA')
    idx = np.argmin(np.abs(info))
    X = X[:,idx]

    estimated_labels = [np.zeros(n) for i in range(T)]    
    ov = np.zeros(T)
    Y = np.array([X[i*n:(i+1)*n] for i in range(T)])

    for i in range(T):
        estimated_labels[i] = ((np.sign(Y[i])+1)/2).astype(int)
        ov[i] = overlap(real_classes[i], estimated_labels[i])


    return np.mean(ov)

def B_opt(AT,n_clusters, Label, eta, lamb):
    
    T = len(AT)
    dT = [np.array(np.sum(AT[i], axis = 0))[0] for i in range(T)]
    n = len(dT[0])
    DT = [scipy.sparse.diags(dT[i], offsets = 0) for i in range(T)]
    
    As = [[None]*T for i in range(T)] # spatial adjacency matrix
    for i in range(T):
        for j in range(T):
            if i == j:
                As[i][j] = AT[i]
            else:
                As[i][j] = None

    As = scipy.sparse.bmat(As)

    Ds = [[None]*T for i in range(T)] # spatial degree matrix
    for i in range(T):
        for j in range(T):
            if i == j:
                Ds[i][j] = DT[i]
            else:
                Ds[i][j] = None

    Ds = scipy.sparse.bmat(Ds)
    I = scipy.sparse.diags(np.ones(n), offsets = 0)

    At = [[None]*T for i in range(T)] # temporal adjacency matrix
    for i in range(T):
        for j in range(T):
            if np.abs(i-j) == 1:
                At[i][j] = I

    At = scipy.sparse.bmat(At)
    Dt = scipy.sparse.diags(np.array(np.sum(At, axis = 0))[0], offsets = 0) # temporal degree matrix

    IT = scipy.sparse.diags(np.ones(T*n), offsets = 0)


    B = scipy.sparse.bmat([[As*lamb,-IT*lamb,As*lamb,None],
                          [(Ds-IT)*lamb, None, Ds*lamb, None],
                          [eta*At, None, eta*At, -eta*IT],
                          [eta*Dt, None, eta*(Dt-IT), None]])


    
    flag = 0
    k = n_clusters
    M = matrix_M(T, lamb, lamb, c, phi, eta)
    mu = np.linalg.eigvals(M).real
    N = matrix_M(T, lamb, 1, c, phi, np.sqrt(eta))
    nu = np.linalg.eigvals(N).real
    counter = np.sum(mu > np.sqrt(np.max(mu))) + np.sum(nu > np.sqrt(np.max(mu)))
    
    v, X = scipy.sparse.linalg.eigs(B, k = counter, which = 'LR')
    
    idx = np.argmin(np.abs(v-np.max(mu)))
    Y = [X[:,idx][i*n:(i+1)*n].real for i in range(T)]    

    ov = np.zeros(T)
    
    for i in range(T):
        estimated_labels = ((np.sign(Y[i])+1)/2).astype(int)
        ov[i] = overlap(Label[i], estimated_labels)

        
    return np.mean(ov)
    start_time

In [ ]:
n = 2500
n_clusters = 2
n_average = 3
fluctuation = 0
fraction = np.ones(n_clusters)/n_clusters
theta = np.ones(n)
phi = 1
n_methods = 3
T = 4 #number of timeslots
c = 6 #average degree
n_step = 15
eta = 0.9
c_out_v = np.linspace(.5,5,n_step)

In [ ]:
t0 = time.time()
ov = np.zeros(((n_methods,n_average,n_step)))
for j in range(n_average):
    for i in range(n_step):
        c_out = c_out_v[i]
        c_in = n_clusters*c - (n_clusters-1)*c_out
        lamb = (c_in - c_out)/(n_clusters*c)
        C = matrix_C(c_out, c, fluctuation, fraction)
        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
        cluster = dynamic_community_detection(AT, eta, real_classes = Label, n_clusters = n_clusters)
        ov[0][j][i] += np.mean(cluster.overlap)
        over, mod = static_BH(AT,n_clusters, Label)
        ov[1][j][i] += np.mean(over)
        ov[2][j][i] += BH_opt(AT,n_clusters, Label, eta, lamb)
        
        OUT = 'i : ' + str(i+1) + '; j : ' + str(j+1) + '; time : ' + str(np.round(time.time()-t0,1))
        sys.stdout.write('\r%s' % OUT)
        sys.stdout.flush()
        
mean = np.zeros((n_methods,n_step))
var = np.zeros((n_methods,n_step))
for i in range(n_methods):
    mean[i] = np.sum(ov[i], axis = 0)/n_average
    var[i] = np.sum(ov[i]**2,axis = 0)/n_average - mean[i]**2

In [ ]:
n = 10000
n_clusters = 2
n_average = 1
fluctuation = 0
fraction = np.ones(n_clusters)/n_clusters
theta = np.ones(n)
phi = 1
n_methods = 3
T = 4 #number of timeslots
c = 6 #average degree
n_step = 15
eta = 0.9
c_out_v = np.linspace(.5,5,n_step)

In [ ]:
t0 = time.time()
n_methods = 2
ov = np.zeros(((n_methods,n_average,n_step)))
for j in range(n_average):
    for i in range(n_step):
        c_out = c_out_v[i]
        c_in = n_clusters*c - (n_clusters-1)*c_out
        lamb = (c_in - c_out)/(n_clusters*c)
        C = matrix_C(c_out, c, fluctuation, fraction)
        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
        alpha = np.sqrt(c)*lamb/find_transition(T,eta)
        if alpha > 1:
            over, mod = dynamic_B(AT, lamb, eta, n_clusters, Label)
            ov[0][j][i] += np.mean(over)
            ov[1][j][i] += B_opt(AT,n_clusters, Label, eta, lamb)

        OUT = 'i : ' + str(i+1) + '; j : ' + str(j+1) + '; time : ' + str(np.round(time.time()-t0,1))
        sys.stdout.write('\r%s' % OUT)
        sys.stdout.flush()
        
        
mean2 = np.zeros((n_methods,n_step))
var2 = np.zeros((n_methods,n_step))

for i in range(n_methods):
    mean2[i] = np.sum(ov[i], axis = 0)/n_average
    var2[i] = np.sum(ov[i]**2,axis = 0)/n_average - mean2[i]**2

In [ ]:
fig = plt.figure(figsize = (14,10))

ax1 = plt.subplot2grid((10,10), (0,0), colspan=5, rowspan=5)
ax2 = plt.subplot2grid((10,10), (0,5), colspan=5, rowspan=5)

ts = 16
fs = 19

ax1.set_ylabel('overlap', fontsize = fs)
ax1.set_xlabel(r'$\alpha/\alpha_c(T,\eta)$', fontsize = fs)
ax1.tick_params(axis="x", labelsize=ts)
ax1.tick_params(axis="y", labelsize=ts)

ax2.tick_params(axis="x", labelsize=ts)
ax2.tick_params(axis="y", labelsize=0)
ax2.set_xlabel(r'$\alpha/\alpha_c(T,\eta)$', fontsize = fs)



marker = ['o', 'p', 'D']
color = ['darkred', 'fuchsia', 'navy']
label1 = ['Alg 1', 'Static BH', r'opt. BH dyn']

color2 = ['darkorange', 'teal']
label2 = ['dyn B', 'opt. dyn B']



ax1.set_ylim(-0.05,1.05)
ax2.set_ylim(-0.05,1.05)

############################################################################

c_in_v = n_clusters*c - (n_clusters-1)*c_out_v
lamb_v = (c_in_v-c_out_v)/(n_clusters*c)
alpha_v = np.sqrt(c)*lamb_v/find_transition(T,eta)


for i in range(3):
    ax1.errorbar(alpha_v,mean[i],np.sqrt(var[i]), marker = marker[i],markersize = 10, fillstyle = 'none', markeredgewidth = 1, color = color[i], linewidth = 2, label = label1[i])
    
ax1.axvline(1, color = 'k', linestyle = '--', linewidth = 0.8)
ax1.legend(fontsize = 14)


############################################################################


for i in range(2):
    ax2.errorbar(alpha_v,mean2[i],np.sqrt(var2[i]), marker = marker[i],markersize = 10, fillstyle = 'none', markeredgewidth = 1, color = color2[i], linewidth = 2, label = label2[i])
    
ax2.axvline(1, color = 'k', linestyle = '--', linewidth = 0.8)
ax2.legend(fontsize = 14, loc = 2)

############################################################################


plt.show();